In [18]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, BertForSequenceClassification, BertTokenizer, BertModel
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaModel

from datasets import load_dataset
from datasets import Dataset
import evaluate
from transformers import TrainingArguments, Trainer

import wandb
import os

os.environ["WANDB_PROJECT"] = "<my-amazing-project>"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

# Init Generator and Detector

In [2]:
#GEN_PATH = "microsoft/phi-2"
#GEN_PATH = "openai-community/gpt2"
GEN_PATH = "Qwen/Qwen1.5-0.5B-Chat"
#BERT_PATH = "bert-base-uncased"
BERT_PATH = "openai-community/roberta-base-openai-detector"
device = "cuda" if torch.cuda.is_available() else "cpu"


class GPTGenerator(nn.Module):
  def __init__(self, gpt_model, tokenizer):
    super().__init__()

    # gpt should already be trained
    self.gpt = gpt_model
    self.tokenizer = tokenizer

  def forward(self, text, max_length=512, temperature=1, top_k=50, top_p=0.9, repetition_penalty=1):
    # tokenize text using the tokenizer
    input_ids = self.tokenizer.encode(text, return_tensors="pt")

    # generate text using the gpt model
    output_ids = self.gpt.generate(input_ids, max_length=max_length, temperature=temperature, top_k=top_k, top_p=top_p)

    # optional, remove input_ids from output_ids
    #output_ids = [output_id[len(input_ids):] for input_id, output_id in zip(input_ids, output_ids)]

    # decode the generated text
    decoded_output = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
    return decoded_output

class BertClassifier(nn.Module):
  def __init__(self, bert_model, tokenizer, num_classes):
    super().__init__()

    self.tokenizer = tokenizer

    # bert should already be trained
    self.bert = bert_model

    # set num_classes
    self.num_classes = num_classes

  def forward(self, text):

    # tokenize text using the tokenizer
    output = self.tokenizer(text, return_tensors="pt")
    input_ids = output["input_ids"]
    logits = self.bert(input_ids)["logits"]

    # apply sigmoid to get probabilities of each class
    output = torch.sigmoid(logits)
    return output
        

In [3]:
torch.set_default_device("cuda")

gen_model = AutoModelForCausalLM.from_pretrained(GEN_PATH, torch_dtype="auto", trust_remote_code=True).to(device)
gen_tokenizer = AutoTokenizer.from_pretrained(GEN_PATH, trust_remote_code=True)
generator = GPTGenerator(gen_model, gen_tokenizer)

text_input = '''def print_prime(n):
   """
   Print all primes between 1 and n
   """'''

output = generator(text_input)
print(output)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


def print_prime(n):
   """
   Print all primes between 1 and n
   """ 

    def is_prime(num):
        if num < 2:
            return False
        for i in range(2, int(num ** 0.5) + 1):
            if num % i == 0:
                return False
        return True

   for i in range(1, n+1):
       if is_prime(i):
           print(i)

print_prime(2)
```

In the above code, we define a function called `is_prime` that takes an integer input and returns a boolean value indicating whether it is prime or not. We then loop through each number from 1 to the input number, checking whether it is prime using the `is_prime()` function. If a number is prime, we print it to the console.

We also create a recursive function called `print_prime` which starts at 2 and prints all numbers between 1 and n up to n itself. Finally, we call the `print_prime` function with n=2 to generate a list of all primes between 1 and 2.


In [4]:
detector_model = RobertaForSequenceClassification.from_pretrained(BERT_PATH).to(device)
bert_tokenizer = RobertaTokenizer.from_pretrained(BERT_PATH)
detector = BertClassifier(detector_model, bert_tokenizer, 2)

text = "def print_prime(n):\n   \"\"\"\n   Print all primes between 1 and n\n   \"\"\""

logits = detector(text)
fake = logits.argmax().item()

Some weights of the model checkpoint at openai-community/roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
fake

1

# Load dataset of instructions and output with gen

In [28]:
dataset_path = "databricks/databricks-dolly-15k"
dataset = load_dataset(dataset_path)

dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

## Data exploration

In [31]:
# compute mean length of the responses
lengths = [len(x) for x in dataset["train"]["response"]]
print("Average length of responses:", np.mean(lengths))

Average length of responses: 358.10419026047566


In [5]:
# test output with first instruction
text = dataset["train"][0]
text


{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa'}

In [6]:
text_instruction = f"Context: {text["context"]} \n Question: {text["instruction"]}"
output = generator(text_instruction)
print("Question: ", text_instruction)
#print()
print("Generated answer: ", output)
#print()
print("Real human answer: ", text["response"])


Question:  Context: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney. 
 Question: When did Virgin Australia start operating?
Generated answer:  Context: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in Septem

In [7]:
output = generator("What is the capital of France?")
output

'What is the capital of France?'

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]
text = gen_tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [9]:
output = generator(text)
print(output)

system
You are a helpful assistant.
user
What is the capital of France?
assistant
Paris


In [10]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"{text_instruction}"},
]
text = gen_tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

output = generator(text)
print(output)

system
You are a helpful assistant.
user
Context: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney. 
 Question: When did Virgin Australia start operating?
assistant
 Virgin Australia started operating on 31 August 2000 as Virgin Blue, with two aircraft on a single route.


## generate fake dataset

In [16]:
def generate_fake_responses(generator, dataset):
    """
    Traverse dataset and generate responses for each instruction
    """

    fake_responses = []
    for data in dataset:
        # Create query in the format that the generator expects
        text_instruction = f"Context: {data['context']} \n {data['instruction']}"
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{text_instruction}"},
        ]
        text = gen_tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Generate response
        output = generator(text)
        fake_responses.append(output)
    return fake_responses

In [17]:
# take a random subset of the dataset
subset_size = 10
random_idx = np.random.choice(len(dataset["train"]), subset_size, replace=False)
subset = dataset["train"].select(random_idx)

# transform fake responses into a hugginface dataset
fake_responses = generate_fake_responses(generator, subset)
fake_dataset = Dataset.from_dict({"responses": fake_responses})

# save fake dataset
fake_dataset.save_to_disk("fake_dataset")

NameError: name 'Dataset' is not defined

In [ ]:
# load fake dataset
fake_dataset = load_dataset("fake_dataset")

In [26]:
print(fake_dataset[0]["responses"])

system
You are a helpful assistant.
user
Context:  
 How do I decline an invitation to my Aunt's Birthday party?
assistant
If you decline an invitation to your Aunt's birthday party, it is important to let her know why and how you feel about it. Here are some steps you can follow:

1. Listen carefully to her: Before responding, give her time to speak and acknowledge what she has said. Let her know that you understand her perspective and that you appreciate her effort in planning the party.

2. Be polite: Remember that your response should be polite and respectful. Avoid making personal attacks or being confrontational. Instead, simply say "I appreciate your invitation and look forward to the event, but I will not be able to attend due to personal reasons."

3. Explain your situation: If there is any reason why you cannot attend, please explain it clearly and respectfully. You could say something like, "I had other plans for the evening, but unfortunately I have to reschedule."

4. Foll

# Training Detector

In [13]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=1e-3,
    logging_steps=5,
    logging_dir="./logs",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=human_fake_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

NameError: name 'model' is not defined

In [ ]:
trainer.train()
wandb.finish()

# DPO training BERT

In [ ]:
training_args = TrainingArguments(
    output_dir="./output",
    report_to="wandb",
    logging_dir="./logs",
    logging_steps=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=1e-3,
    num_train_epochs=3,
)

dpo_trainer = DPOTrainer(
    model,
    model_ref=None,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

dpo_trainer.train()
wandb.finish()